[View in Colaboratory](https://colab.research.google.com/github/whmay/capstone/blob/master/landmark.ipynb)

In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 18408 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Selecting previously unselected package dirmngr.
Preparing to unpack .../04-dirmngr_2.1.15-1ubuntu8.1_amd64.deb ...
Unpacking dirmngr (2.1.15-1ubuntu8.1) ...
Selecting previously unselected package distro-info-data.
Preparing to unpack .

In [0]:
!pip install -U -q PyDrive
# Generate auth tokens for Colab
from google.colab import auth
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
# Generate creds for the Drive FUSE library.
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
# Create a directory and mount Google Drive using that directory.
!mkdir -p drive
!google-drive-ocamlfuse drive

print('Files in Drive:')
!ls drive/


In [5]:
!pip install keras
!pip install h5py

In [0]:
#!pip install -U --force-reinstall --no-dependencies git+https://github.com/datumbox/keras@bugfix/trainable_bn
#http://blog.datumbox.com/the-batch-normalization-layer-of-keras-is-broken/

In [6]:
#check whether you are using GPU
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [7]:
!ls drive/Resnet2
!cat drive/Resnet2/labels.txt


labels.txt  test  train
鸟巢（国家体育场）
颐和园
雍和宫
水立方（国家游泳中心）
毛主席纪念堂
桃坪羌寨
树正瀑布
松潘古城
恭王府
居庸关长城
天安门广场
圆明园
北海公园
前门大街
凤凰岭自然风景区
八达岭长城
什刹海
人民英雄纪念碑
人民大会堂
五彩池
争艳池
九曲黄河第一湾
中国国家博物馆

In [8]:
!pip install -U coremltools

    100% |████████████████████████████████| 2.5MB 8.2MB/s 
  Found existing installation: six 1.11.0
    Uninstalling six-1.11.0:
      Successfully uninstalled six-1.11.0


In [9]:
from keras.applications.resnet50 import ResNet50
from keras.models import Model,load_model
from keras.layers import Dense,GlobalAveragePooling2D,Input
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adadelta
import keras
import math, os, sys
import matplotlib.pyplot as plt
import coremltools



def get_model():
  
    input_tensor = Input(shape=(224, 224, 3))  # this assumes K.image_data_format() == 'channels_last'

    # create the base pre-trained model
    base_model = ResNet50(input_tensor=input_tensor,weights='imagenet',include_top=False)
    
    #base_model = ResNet50(input_tensor=None,weights='imagenet',include_top=False)

    for layer in base_model.layers:
        layer.trainable=False

    x = base_model.output
    x = GlobalAveragePooling2D(data_format='channels_last')(x)
    x = Dense(num_classes, activation='softmax')(x)

    updatedModel = Model(base_model.input, x)

    return  updatedModel



def compile_model(compiledModel):

    compiledModel.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=Adadelta(),
                  metrics=['accuracy'])


def modelFitGenerator(fitModel):

    num_train_samples = sum([len(files) for r, d, files in os.walk(train_data_dir)])
    num_valid_samples = sum([len(files) for r, d, files in os.walk(validation_data_dir)])

    num_train_steps = math.floor(num_train_samples/batch_size)
    num_valid_steps = math.floor(num_valid_samples/batch_size)
    
    train_datagen = ImageDataGenerator(  
      rotation_range=90,      
      horizontal_flip=True,    
      vertical_flip=True,
      zoom_range=0.4)

    test_datagen = ImageDataGenerator()

    train_generator = train_datagen.flow_from_directory(
      train_data_dir,
      target_size=image_size ,
      batch_size=batch_size,
      class_mode='categorical', shuffle=True
    )

    validation_generator = test_datagen.flow_from_directory(
      validation_data_dir,
      target_size=image_size ,
      batch_size=batch_size,
      class_mode='categorical', shuffle=True
    )

    print("start history model")
    history = fitModel.fit_generator(
      train_generator,
      steps_per_epoch=num_train_steps,
      epochs=nb_epoch,
      validation_data=validation_generator,
      validation_steps=num_valid_steps)
    
    printGraph(history)

def printGraph(history):
    
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
     
    
# def main():
#     model = get_model()
#     compile_model(model)
#     modelFitGenerator(model)
#     saveCoreMLModel(model)


# if __name__ == '__main__':
#     # constants
#     image_size = (224, 224)
#     train_data_dir = 'drive/Resnet/landmark/train' 
#     validation_data_dir = 'drive/Resnet/landmark/test'
#     nb_epoch = 50
#     batch_size = 16
#     num_classes = 2
#     main()


Using TensorFlow backend.


In [0]:
image_size = (224, 224)
train_data_dir = 'drive/Resnet2/train' 
validation_data_dir = 'drive/Resnet2/test'
nb_epoch = 50
batch_size = 16
num_classes = 23

In [0]:
model = get_model()
compile_model(model)
modelFitGenerator(model)

In [12]:
def saveCoreMLModel(kerasModel):
    coreml_model = coremltools.converters.keras.convert(kerasModel,
                                                    input_names=['input'],
                                                    output_names=['probs'],
                                                    image_input_names='input',
                                                    predicted_feature_name='predictedMoney',
                                                    class_labels = 'drive/Resnet2/labels.txt')
    coreml_model.save('resnet50custom.mlmodel') 
    print('CoreML model saved')
    
saveCoreMLModel(model)

0 : input_1, <keras.engine.topology.InputLayer object at 0x7f751a52c7b8>
1 : conv1_pad, <keras.layers.convolutional.ZeroPadding2D object at 0x7f751a52c978>
2 : conv1, <keras.layers.convolutional.Conv2D object at 0x7f751a52c390>
3 : bn_conv1, <keras.layers.normalization.BatchNormalization object at 0x7f751a52cbe0>
4 : activation_1, <keras.layers.core.Activation object at 0x7f751a52ce48>
5 : max_pooling2d_1, <keras.layers.pooling.MaxPooling2D object at 0x7f74fd27c860>
6 : res2a_branch2a, <keras.layers.convolutional.Conv2D object at 0x7f751a5218d0>
7 : bn2a_branch2a, <keras.layers.normalization.BatchNormalization object at 0x7f74fba29a20>
8 : activation_2, <keras.layers.core.Activation object at 0x7f74fb106748>
9 : res2a_branch2b, <keras.layers.convolutional.Conv2D object at 0x7f74fb0a9d68>
10 : bn2a_branch2b, <keras.layers.normalization.BatchNormalization object at 0x7f74fb0a9748>
11 : activation_3, <keras.layers.core.Activation object at 0x7f74fb087b38>
12 : res2a_branch2c, <keras.layer

In [0]:
from google.colab import files
files.download('resnet50custom.mlmodel') 

In [13]:
uploaded = drive.CreateFile()
uploaded.SetContentFile('resnet50custom.mlmodel')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.locked_file'

During handling of the above exception, another exceptio

Uploaded file with ID 1BrwmyeBYqNC79CsugmiWzo1Y1d6ylr8Y
